<div style="line-height:1.2;">

<h1 style="color:#FF7C00; margin-bottom: 0.3em;">Tensorflow basics 2 </h1>

<h4 style="margin-top: 0.3em; margin-bottom: 1em;"> Other Tensors and Neural Networks examples.</h4>

<div style="line-height:1.4; margin-bottom: 0.5em;">
    <h3 style="color: lightblue; display: inline; margin-right: 0.5em;">Keywords:</h3> 
    jedi completion in Colab + tf.dtype + @tf.function decorator + tf.math + tf.constant
</div>

</div>

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
%config Completer.use_jedi = True
import numpy as np
import tensorflow as tf

In [4]:
# Check GPU
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
!nvidia-smi 

Thu Jul  6 14:39:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
one_tens = tf.constant(np.arange(1, 10))
one_tens

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [16]:
dtype_of_elements_in_one_tens = one_tens.dtype
dtype_of_elements_in_one_tens

tf.float32

In [7]:
squa = tf.square(one_tens)
squa

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81])>

### => sqrt, take the square root of each tensor's value

In [8]:
""" sqrt 1 
N.B.
sq = tf.sqrt(one_tens) #Error! --> InvalidArgumentError: allowed values: bfloat16, half, float, double, complex64, complex128
"""
## To use the same one_tens
casted = tf.cast(one_tens, dtype=tf.float32)
tf.sqrt(casted)
#print(casted)

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [9]:
""" sqrt 2. Create same tensor in different way"""

another_tens = tf.constant(np.arange(1, 10, dtype=np.float32))

sq = tf.sqrt(another_tens)
print(another_tens)
print(sq)

tf.Tensor([1. 2. 3. 4. 5. 6. 7. 8. 9.], shape=(9,), dtype=float32)
tf.Tensor(
[1.        1.4142135 1.7320508 2.        2.236068  2.4494898 2.6457512
 2.828427  3.       ], shape=(9,), dtype=float32)


In [17]:
dtype_of_elements_in_sq = sq.dtype
dtype_of_elements_in_sq

tf.float32

### => log

In [14]:
""" Find the log 
N.B.
Using 'tf.math.log(one_tens)' leads to error: InvalidArgumentError.
The tensor need to contains floats! 
"""
# Convert to float32
one_tens_float = tf.cast(one_tens, dtype=tf.float32)  
one_tens_float

<tf.Tensor: shape=(9,), dtype=float32, numpy=array([1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32)>

In [15]:
log_tens = tf.math.log(one_tens_float)
log_tens

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

<h2 style="color:#FF7C00  "> Manipulate tf.Variable </h2>

In [18]:
ten2 = tf.Variable(np.arange(0, 5))
ten2

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([0, 1, 2, 3, 4])>

In [22]:
# Show only the content 
print(ten2.numpy())

[ 0  1  2  3 50]


In [21]:
## Assign the final value a new value of 50
ten2.assign([0, 1, 2, 3, 50])
ten2

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([ 0,  1,  2,  3, 50])>


In [20]:
## Add 10 to every element in I
ten2.assign_add([10, 10, 10, 10, 10])
ten2

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([10, 11, 12, 13, 60])>

<h3 style="color:#FF7C00  "> Tf with Numpy </h3>

In [19]:
## Tensor from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [20]:
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [21]:
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [22]:
## Create a tensor from NumPy and from an array
numpy_J = tf.constant(np.array([3., 7., 10.]))  #float64
tensor_J = tf.constant([3., 7., 10.])           #float32
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

<h2 style="color:#FF7C00  "> decorator @tf.function </h2>

In [10]:
def func(x, y):
  return x ** 2 + y

x = tf.constant(np.arange(0, 10))
y = tf.constant(np.arange(10, 20))
func(x, y)

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 10,  12,  16,  22,  30,  40,  52,  66,  82, 100])>

In [11]:
## Same function and decorate it with tf.function
@tf.function
def tf_function(x, y):
  return x ** 2 + y

tf_function(x, y)

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 10,  12,  16,  22,  30,  40,  52,  66,  82, 100])>

In [12]:
""" Function that will be converted to a graph function. """
@tf.function
def add(a, b):
    return tf.add(a, b)

a = tf.constant([1, 2, 3])
b = tf.constant([4, 5, 6])

## Graph function
c = add(a, b)
c

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([5, 7, 9], dtype=int32)>

In [24]:
real = np.random.rand(3, 3)
imag = np.random.rand(3, 3)
complex_tens = tf.complex(real, imag)

# compute the absolute value of the complex tensor
abs_tens = tf.abs(complex_tens)
# compute the conjugate of the complex tensor
conj_tens = tf.math.conj(complex_tens)

print(real)
print(imag)
print()
print(abs_tens)
print(conj_tens)

[[0.47567106 0.27498809 0.27722176]
 [0.56314534 0.72622029 0.94351461]
 [0.88454316 0.62311073 0.39087869]]
[[0.15281473 0.67864045 0.31582384]
 [0.04482452 0.9358446  0.94483731]
 [0.78597771 0.55608315 0.75109824]]

tf.Tensor(
[[0.49961515 0.7322372  0.42023398]
 [0.56492646 1.18456786 1.33526678]
 [1.18329099 0.83516193 0.84671997]], shape=(3, 3), dtype=float64)
tf.Tensor(
[[0.47567106-0.15281473j 0.27498809-0.67864045j 0.27722176-0.31582384j]
 [0.56314534-0.04482452j 0.72622029-0.9358446j  0.94351461-0.94483731j]
 [0.88454316-0.78597771j 0.62311073-0.55608315j 0.39087869-0.75109824j]], shape=(3, 3), dtype=complex128)
